In [1]:
from attention import AttentionLayer
import numpy as np  
import pandas as pd 
import re           
from bs4 import BeautifulSoup 
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords   
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from keras import backend as K 
#import tensorflow as tf
# physical_devices = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

#gpu_options = tf.GPUOptions(allow_growth=True)
#session = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

In [2]:
%time
dataset_base = pd.read_csv("dataset_base.csv", sep='\t', encoding='utf-8')
dataset_base.sample(5)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 4.53 µs


,category,title,date,abstract,content,content_for_topic,content_for_summary,abstract_for_summary
2651,SEGURO INTEGRAL DE SALUD,RESOLUCION JEFATURAL N° 076-2021/SIS,22/06/2021,Aprueban la actualización de la Directiva Administrativa N° 002-2020-SIS/GNF-V.01 la que se denominará: Directiva Administrativa N° 001-2021-SIS/GNF-V.02 Directiva Administrativa para el Control F...,"Lima, 21 de junio de 2021 VISTOS: El Informe Conjunto N° 001-2021-SIS/GNF-SGRF-SGF/DMEG-JGMN-JJCM-KERA-RELS-VICA con Proveído N° 329-2021-SIS/GNF e Informe Conjunto N° 002-2021-SIS/GNF-SGRF-SGF/DM...",lima junio visto informe conjunto sgrf dmeg jgmn jjcm kero rels vico proveído informe conjunto sgrf dmeg jgmn jjcm kero rels vico proveído gerencia negocio financiamiento informe ogppdo aamz prove...,lima junio vistos informe conjunto sgrf dmeg jgmn jjcm kera rels vica proveído informe conjunto sgrf dmeg jgmn jjcm kera rels vica proveído gerencia negocios financiamiento informe ogppdo aamz pro...,aprueban actualización directiva administrativa denominará directiva administrativa directiva administrativa control financiero procedimientos monitoreo supervisión seguimiento trasferencias finan...
1853,MUNICIPALIDAD DE SANTA ANITA,DECRETO DE ALCALDIA N° 000005-2021/MDSA,08/07/2021,Prorrogan vigencia de la Ordenanza N° 000303/MDSA que aprobó la Amnistía General Tributaria 2021,"Santa Anita, 30 de junio de 2021 EL ALCALDE DE LA MUNICIPALIDAD DISTRITAL DE SANTA ANITA VISTO: El Informe Nº 0136-2021-SGRCRT/GSAT/MDSA de la Subgerencia de Registro, Control y Recaudación Tribut...",santo anito junio alcalde municipalidad distrital santa anito visto informe sgrcrt gsat mdsa subgerencia registro control recaudación tributario memorándum gsat mdsa gerencia servicio administraci...,santa anita junio alcalde municipalidad distrital santa anita visto informe sgrcrt gsat mdsa subgerencia registro control recaudación tributaria memorándum gsat mdsa gerencia servicios administrac...,prorrogan vigencia ordenanza mdsa aprobó amnistía general tributaria
654,ORGANISMO DE EVALUACION Y FISCALIZACION AMBIENTAL,RESOLUCION N° 00040-2021-OEFA/PCD,28/07/2021,Autorizan Transferencia Financiera a favor del Ministerio del Ambiente - MINAM Instituto de Investigaciones de la Amazonía Peruana - IIAP Instituto Geofísico del Perú - IGP y del Servicio Nacional...,"Lima, 27 de julio de 2021 VISTOS: El Oficio Múltiple N° 00020-2021-MINAM/SG/OGPP, emitido por la Oficina General de Planeamiento y Presupuesto del Ministerio del Ambiente; el Informe N° 00150-2021...",lima julio visto oficio múltiple minam ogpp emitido oficina general planeamiento presupuesto ministerio ambiente informe oefar emitido oficina planeamiento presupuesto informe oefar emitido oficin...,lima julio vistos oficio múltiple minam ogpp emitido oficina general planeamiento presupuesto ministerio ambiente informe oefa emitido oficina planeamiento presupuesto informe oefa emitido oficina...,autorizan transferencia financiera favor ministerio ambiente minam instituto investigaciones amazonía peruana iiap instituto geofísico perú servicio nacional áreas protegidas sernanp
1088,MUNICIPALIDAD METROPOLITANA DE LIMA,ORDENANZA N° 2376-2021,23/07/2021,Ordenanza que modifica el plano de zonificacion del distrito de Ate aprobado por Ordenanza N° 1099,"Lima, 16 de julio de 2021 EL TENIENTE ALCALDE ENCARGADO DEL DESPACHO DE ALCALDÍA POR CUANTO: El Concejo Metropolitano de Lima, en Sesión Ordinaria de la fecha; CONSIDERANDO: Que, el Artículo 194 d...",lima julio teniente alcalde encargado despacho alcaldía concejo metropolitano limo sesión ordinario fecha constitución político perú concordancia título preliminar orgánico municipalidad establece...,lima julio teniente alcalde encargado despacho alcaldía concejo metropolitano lima sesión ordinaria fecha constitución política perú concordancia título preliminar orgánica municipalidades estable...,ordenanza modifica plano zonificacion distrito aprobado ordenanza
3889,MUNICIPALIDAD DE LOS OLIVOS,RESOLUCION 

In [3]:
# Limpieza del dataset
print("Dimensiones:", dataset_base.shape)
print("Filas que no contiene valores nulos:", len(dataset_base[dataset_base.content_for_topic.notna()]))
print("Filas con valores nulos:", len(dataset_base[dataset_base.isnull().any(axis=1)]))

Dimensiones: (4163, 8)
Filas que no contiene valores nulos: 4126
Filas con valores nulos: 37


In [4]:
# Eliminado los que contien valores nulos
dataset_base.dropna(inplace=True)
print("Dimensiones:", dataset_base.shape)

Dimensiones: (4126, 8)


In [5]:
max_len_text=80 
max_len_summary=20

In [6]:
x_tr,x_val,y_tr,y_val=train_test_split(dataset_base.content_for_summary,dataset_base.abstract_for_summary,test_size=0.1,random_state=0,shuffle=True) 

In [7]:
%%time
x_tokenizer = Tokenizer()
x_tokenizer.fit_on_texts(list(x_tr))

#convert text sequences into integer sequences
x_tr    =   x_tokenizer.texts_to_sequences(x_tr) 
x_val   =   x_tokenizer.texts_to_sequences(x_val)

#padding zero upto maximum length
x_tr    =   pad_sequences(x_tr,  maxlen=max_len_text, padding='post') 
x_val   =   pad_sequences(x_val, maxlen=max_len_text, padding='post')

x_voc_size   =  len(x_tokenizer.word_index) +1

CPU times: user 5.19 s, sys: 29.4 ms, total: 5.22 s
Wall time: 5.22 s


In [8]:
%%time
#preparing a tokenizer for summary on training data 
y_tokenizer = Tokenizer()
y_tokenizer.fit_on_texts(list(y_tr))

#convert summary sequences into integer sequences
y_tr    =   y_tokenizer.texts_to_sequences(y_tr) 
y_val   =   y_tokenizer.texts_to_sequences(y_val) 

#padding zero upto maximum length
y_tr    =   pad_sequences(y_tr, maxlen=max_len_summary, padding='post')
y_val   =   pad_sequences(y_val, maxlen=max_len_summary, padding='post')

y_voc_size  =   len(y_tokenizer.word_index) +1

CPU times: user 161 ms, sys: 170 µs, total: 161 ms
Wall time: 159 ms


In [9]:
K.clear_session() 
latent_dim = 500 

# Encoder 
encoder_inputs = Input(shape=(max_len_text,)) 
enc_emb = Embedding(x_voc_size, latent_dim,trainable=True)(encoder_inputs) 

#LSTM 1 
encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True) 
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb) 

#LSTM 2 
encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True) 
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1) 

#LSTM 3 
encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True) 
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2) 

# Set up the decoder. 
decoder_inputs = Input(shape=(None,)) 
dec_emb_layer = Embedding(y_voc_size, latent_dim,trainable=True) 
dec_emb = dec_emb_layer(decoder_inputs) 

#LSTM using encoder_states as initial state
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True) 
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c]) 

#Attention Layer
attn_layer = AttentionLayer(name='attention_layer') 
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs]) 

# Concat attention output and decoder LSTM output 
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

#Dense layer
decoder_dense = TimeDistributed(Dense(y_voc_size, activation='softmax')) 
decoder_outputs = decoder_dense(decoder_concat_input) 

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs) 
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 80)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 80, 500)      19700000    input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 80, 500), (N 2002000     embedding[0][0]                  
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
______________________________________________________________________________________________

In [10]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

In [11]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

In [12]:
history=model.fit([x_tr,y_tr[:,:-1]], y_tr.reshape(y_tr.shape[0],y_tr.shape[1], 1)[:,1:] ,epochs=50,callbacks=[es],batch_size=512, validation_data=([x_val,y_val[:,:-1]], y_val.reshape(y_val.shape[0],y_val.shape[1], 1)[:,1:]))

Epoch 1/50


ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted:  OOM when allocating tensor with shape[9728,5911] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model/time_distributed/dense/MatMul (defined at <ipython-input-12-93aee35d4b44>:1) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[gradient_tape/model/embedding/embedding_lookup/Reshape/_114]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted:  OOM when allocating tensor with shape[9728,5911] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model/time_distributed/dense/MatMul (defined at <ipython-input-12-93aee35d4b44>:1) ]]
Hint: If you want to see a list of allocated tensors when OOM hap